# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.5205643870100998)),
 (1, np.float64(2.5205643870100998)),
 (2, np.float64(2.5205643870100998)),
 (3, np.float64(2.5205643870100998)),
 (4, np.float64(2.5205643870100998))]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('a', 2)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.007935156173640268)),
   (None, np.float64(0.0567127676575484)),
   (None, np.float64(0.05804094881567723)),
   (None, np.float64(0.07045657315391529)),
   (None, np.float64(0.07087191057870468)),
   (None, np.float64(0.07177667962025525)),
   (None, np.float64(0.11980410104791872)),
   (None, np.float64(0.14239735951436605)),
   (None, np.float64(0.15543862293346278)),
   (None, np.float64(0.1633813817799833)),
   (None, np.float64(0.16893798558703177)),
   (None, np.float64(0.2211883306209127)),
   (None, np.float64(0.23109687499339437)),
   (None, np.float64(0.23180967766202598)),
   (None, np.float64(0.26369939320119284)),
   (None, np.float64(0.31610653245295006)),
   (None, np.float64(0.34656724287396345)),
   (None, np.float64(0.42562733623107585)),
   (None, np.float64(0.4494287787240133)),
   (None, np.float64(0.4646252660529876)),
   (None, np.float64(0.4742327556164522)),
   (None, np.float64(0.4881313552416081)),
   (None, np.float64(0.4994481615

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [30]:
input=np.random.randint(-100,1000,size=50)
def RECORDREADER():
  return [(i,val) for i,val in enumerate(input)]

def MAP(key,val):
  yield ("max",val)

def REDUCE(key,values):
  max=-np.inf
  for val in values:
    if val>max:
      max=val
  yield (key,max)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
print(f"Output: {output}")
print(f"Numpy: {np.max(input)}")

Output: [('max', np.int64(985))]
Numpy: 985


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [31]:
input=np.random.randint(-100,1000,size=50)
def RECORDREADER():
  return [(i,val) for i,val in enumerate(input)]

def MAP(key,val):
  yield ("avg",val)

def REDUCE(key,values):
  s=0
  count=0
  for val in values:
    s+=val
    count+=1
  s/=count
  yield (key,s)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
print(f"Output: {output}")
print(f"Numpy: {np.mean(input)}")

Output: [('avg', np.float64(452.54))]
Numpy: 452.54


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [32]:
a=((3,7),(1,17),(8,15),(1,4),(5,6),(3,22),(8,7))
def groupbykey_sort(iterable):
  new_iterable=sorted(iterable,key=lambda x:x[0])
  res={}
  cur_key=new_iterable[0][0]
  arr=[]
  for k,v in new_iterable:
    if k==cur_key: arr.append(v)
    else:
      res[cur_key]=arr
      arr=[v]
      cur_key=k
  res[cur_key]=arr
  return res

groupbykey_sort(a)

{1: [17, 4], 3: [7, 22], 5: [6], 8: [15, 7]}

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [33]:
input = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=3, age=33, gender="female", social_contacts=800),
    User(id=4, age=21, gender="male", social_contacts=100),
    User(id=4, age=21, gender="male", social_contacts=100)
]
maps = 3
reducers = 3

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for val in split:
      yield (val,1)

  split_size =  int(np.ceil(len(input)/maps))
  for i in range(0, len(input), split_size):
    yield RECORDREADER(input[i:i+split_size])

def MAP(val,num):
  yield (val, 1)

def REDUCE(val, numbers:Iterator[int]):
  yield (None, val)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

7 key-value pairs were sent over a network.


[(0, [(None, User(id=3, age=33, social_contacts=800, gender='female'))]),
 (1,
  [(None, User(id=1, age=25, social_contacts=240, gender='female')),
   (None, User(id=4, age=21, social_contacts=100, gender='male'))]),
 (2,
  [(None, User(id=0, age=55, social_contacts=20, gender='male')),
   (None, User(id=2, age=25, social_contacts=500, gender='female'))])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [35]:
input = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]
def RECORDREADER():
  return [(i,row) for i,row in enumerate(input)]

def MAP(_,row):
  if(row.age>30):
    yield (row,row)

def REDUCE(key,values):
  yield (key,values)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  [User(id=0, age=55, social_contacts=20, gender='male')]),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  [User(id=3, age=33, social_contacts=800, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [36]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]
def RECORDREADER():
  return [(i,row) for i,row in enumerate(input_collection)]

def MAP(_,row):
  S=['age','social_contacts']
  t=tuple([row[row._fields.index(field)] for field in S])
  yield (t,t)

def REDUCE(key,values):
  yield (key,values[0])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
output

[((55, 20), (55, 20)),
 ((25, 240), (25, 240)),
 ((25, 500), (25, 500)),
 ((33, 800), (33, 800))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [37]:
input_collection1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]
input_collection2 = [
    User(id=5, age=50, gender='male', social_contacts=45),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=6, age=40, gender='female', social_contacts=300)
]
def RECORDREADER():
  return [(i,row) for i,row in enumerate(input_collection1+input_collection2)]

def MAP(_,row):
  yield (row,row)

def REDUCE(key,values):
  yield (key,values[0])
output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=5, age=50, social_contacts=45, gender='male'),
  User(id=5, age=50, social_contacts=45, gender='male')),
 (User(id=6, age=40, social_contacts=300, gender='female'),
  User(id=6, age=40, social_contacts=300, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [38]:
input_collection1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]
input_collection2 = [
    User(id=5, age=50, gender='male', social_contacts=45),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=6, age=40, gender='female', social_contacts=300)
]
def RECORDREADER():
  return [(i,row) for i,row in enumerate(input_collection1+input_collection2)]

def MAP(_,row):
  yield (row,row)

def REDUCE(key,values):
  if len(values)==2:
    yield (key,values[0])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
output

[(User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [39]:
input_collection1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]
input_collection2 = [
    User(id=5, age=50, gender='male', social_contacts=45),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=6, age=40, gender='female', social_contacts=300)
]
def RECORDREADER():
  return [(1,row) for row in input_collection1]+[(2,row) for row in input_collection2]

def MAP(name,row):
  yield (row,name)

def REDUCE(key,values):
  if (values==[1]):
    yield (key,values[0])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'), 1),
 (User(id=2, age=25, social_contacts=500, gender='female'), 1),
 (User(id=3, age=33, social_contacts=800, gender='female'), 1)]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [40]:
class Order(NamedTuple):
  order_id: int
  date: str
class Order_item(NamedTuple):
  order_id: int
  item_name: str

orders=[
    Order(order_id=1,date='01.04.2025'),
    Order(order_id=2,date='15.04.2025')
]
items=[
    Order_item(order_id=1,item_name='laptop'),
    Order_item(order_id=1,item_name='joystick'),
    Order_item(order_id=2,item_name='headphones')
]
def RECORDREADER():
  return [('order',row) for row in orders]+[('item',row) for row in items]

def MAP(name,row):
  a=row.date if name=='order' else row.item_name
  yield (row.order_id,(name,a))

def REDUCE(key,values):
  a_name=values[0][0]
  a_array=[]
  c_array=[]
  for val in values:
    if val[0]==a_name: a_array.append(val[1])
    else: c_array.append(val[1])
  for a in a_array:
    for c in c_array:
      yield (None,(a,key,c))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
output

[(None, ('01.04.2025', 1, 'laptop')),
 (None, ('01.04.2025', 1, 'joystick')),
 (None, ('15.04.2025', 2, 'headphones'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [41]:
class Order(NamedTuple):
  order_id: int
  date: str
  sum: int

orders=[
    Order(order_id=1,date='01.04.2025',sum=30),
    Order(order_id=2,date='15.04.2025',sum=10),
    Order(order_id=3,date='15.04.2025',sum=25),
    Order(order_id=4,date='01.04.2025',sum=10)
]
def RECORDREADER():
  return [(None,row) for row in orders]

def MAP(_,row):
  yield (row.date,row.sum)

def REDUCE(key,values):
  yield (key,sum(values))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
output

[('01.04.2025', 40), ('15.04.2025', 35)]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [42]:
import numpy as np
vector=[1,2,3,4]
matrix=[[5,6,7,8],
        [9,10,11,12]]

def RECORDREADER():
  return [(idx,val) for idx,val in enumerate(vector)]

def MAP(idx,val):
  for num,row in enumerate(matrix):
    yield (num,val*row[idx])

def REDUCE(num,values):
  yield (num,sum(values))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output=list(output)
print(f"Output: {output}")
print(f"Numpy result: {np.array(matrix)@np.array(vector)}")

Output: [(0, 70), (1, 110)]
Numpy result: [ 70 110]


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [43]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [44]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i,row in enumerate(small_mat):
    yield((i,k),row[j]*w)

def REDUCE(key, values):
  (i, k) = key
  res=np.sum(values)
  yield((i,k),res)

Проверьте своё решение

In [45]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [46]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((1,i,j), small_mat[i,j])
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((2,j,k), big_mat[j,k])

def MAP(k1, v1):
  if (k1[0]==1):
    for k in range(K):
      yield((k1[1],k),(k1[2],v1))
  else:
    for i in range(I):
      yield((i,k1[2]),(k1[1],v1))

def REDUCE(key, values):
  (i, k) = key
  sorted_values=sorted(values,key=lambda x:x[0])
  res=0
  for n in range(int(len(sorted_values)/2)):
    v1=sorted_values[2*n][1]
    v2=sorted_values[2*n+1][1]
    res+=v1*v2
  yield((i,k),res)



In [47]:
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [48]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
maps = 3
reducers = 3

def INPUTFORMAT():
  global maps

  def RECORDREADER(mat,num):
    for i in range(mat.shape[0]):
      for j in range(mat.shape[1]):
        yield ((num,i,j), mat[i,j])

  yield RECORDREADER(small_mat,1)
  yield RECORDREADER(big_mat,2)

def MAP(k1, v1):
  if (k1[0]==1):
    for k in range(K):
      yield((k1[1],k),(k1[2],v1))
  else:
    for i in range(I):
      yield((i,k1[2]),(k1[1],v1))

def REDUCE(key, values):
  (i, k) = key
  sorted_values=sorted(values,key=lambda x:x[0])
  res=0
  for n in range(int(len(sorted_values)/2)):
    v1=sorted_values[2*n][1]
    v2=sorted_values[2*n+1][1]
    res+=v1*v2
  yield((i,k),res)

reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
solution= [(partition_id, list(partition)) for (partition_id, partition) in solution]

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  res_list=[]
  for item in reduce_output:
    res_list+=item[1]
  I = max([i for ((i,k), vw) in res_list])+1
  K = max([k for ((i,k), vw) in res_list])+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in res_list:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

480 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [49]:
I = 10
J = 3*5
K = 4*5
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
maps = 3
reducers = 3

def INPUTFORMAT():
  global maps

  def RECORDREADER(rows,row_num,mat_num):
    for i in range(rows.shape[0]):
      for j in range(rows.shape[1]):
        yield ((mat_num,i+row_num,j), rows[i,j])

  split_size =  int(np.ceil(small_mat.shape[0]/maps))
  for i in range(0, small_mat.shape[0], split_size):
    yield RECORDREADER(small_mat[i:i+split_size,:],i,1)
  split_size =  int(np.ceil(big_mat.shape[0]/maps))
  for i in range(0, big_mat.shape[0], split_size):
    yield RECORDREADER(big_mat[i:i+split_size,:],i,2)

def MAP(k1, v1):
  if (k1[0]==1):
    for k in range(K):
      yield((k1[1],k),(k1[2],v1))
  else:
    for i in range(I):
      yield((i,k1[2]),(k1[1],v1))

def REDUCE(key, values):
  (i, k) = key
  sorted_values=sorted(values,key=lambda x:x[0])
  res=0
  for n in range(int(len(sorted_values)/2)):
    v1=sorted_values[2*n][1]
    v2=sorted_values[2*n+1][1]
    res+=v1*v2
  yield((i,k),res)

reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
solution= [(partition_id, list(partition)) for (partition_id, partition) in solution]

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  res_list=[]
  for item in reduce_output:
    res_list+=item[1]
  I = max([i for ((i,k), vw) in res_list])+1
  K = max([k for ((i,k), vw) in res_list])+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in res_list:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

6000 key-value pairs were sent over a network.


True

При генерации случайного подмножества элементов матрицы решение должно работать, так как каждый сгенерированный элемент будет определятся номером матрицы и его координатами в матрице.